<a href="https://colab.research.google.com/github/ihsanramdhani/skin-cancer-detection/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls "/content/drive/MyDrive/kaggle API Token"

kaggle.json


In [78]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp "/content/drive/MyDrive/kaggle API Token/kaggle.json" ~/.kaggle/
!cat ~/.kaggle/kaggle.json 
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d fanconic/skin-cancer-malignant-vs-benign -p /content/drive/MyDrive/dataset

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [34]:
import os

try:
  os.mkdir("/content/drive/MyDrive/data")
  os.mkdir("/content/drive/MyDrive/raw_data")

  os.mkdir("/content/drive/MyDrive/data/training")
  os.mkdir("/content/drive/MyDrive/data/validation")
  os.mkdir("/content/drive/MyDrive/data/testing")

  os.mkdir("/content/drive/MyDrive/data/training/benign")
  os.mkdir("/content/drive/MyDrive/data/training/malignant")

  os.mkdir("/content/drive/MyDrive/data/validation/benign")
  os.mkdir("/content/drive/MyDrive/data/validation/malignant")

  os.mkdir("/content/drive/MyDrive/data/testing/benign")
  os.mkdir("/content/drive/MyDrive/data/testing/malignant")

  os.mkdir("/content/drive/MyDrive/source")
  os.mkdir("/content/drive/MyDrive/source/benign")
  os.mkdir("/content/drive/MyDrive/source/malignant")
except:
  print("Failed to create the data, check !ls to confirm.")
  pass


In [14]:
import shutil 

shutil.unpack_archive("/content/drive/MyDrive/dataset/skin-cancer-malignant-vs-benign.zip", "/content/drive/MyDrive/raw_data")

In [73]:
print("Total Testing Data:")
print(len(os.listdir("/content/drive/MyDrive/raw_data/test/benign")))
print(len(os.listdir("/content/drive/MyDrive/raw_data/test/malignant")))

print("Total Training Data")
print(len(os.listdir("/content/drive/MyDrive/raw_data/train/benign")))
print(len(os.listdir("/content/drive/MyDrive/raw_data/train/malignant")))

Total Testing Data:
360
300
Total Training Data
1440
1197


In [71]:
import os
import glob
import shutil


def copy_data(src, dst):
  for f in glob.iglob(os.path.join(src, "*.jpg")):
    shutil.copy(f, dst)

TRAIN_BENIGN_DIR = "/content/drive/MyDrive/raw_data/train/benign"
TRAIN_MALIGNANT_DIR = "/content/drive/MyDrive/raw_data/train/benign"
TEST_BENIGN_DIR = "/content/drive/MyDrive/raw_data/test/benign"
TEST_MALIGNANT_DIR = "/content/drive/MyDrive/raw_data/test/malignant"
DEST_BENIGN_DIR = "/content/drive/MyDrive/source/benign"
DEST_MALIGNANT_DIR = "/content/drive/MyDrive/source/malignant"

copy_data(TRAIN_BENIGN_DIR, DEST_BENIGN_DIR)
copy_data(TEST_BENIGN_DIR, DEST_BENIGN_DIR)
copy_data(TRAIN_MALIGNANT_DIR, DEST_MALIGNANT_DIR)
copy_data(TEST_MALIGNANT_DIR, DEST_MALIGNANT_DIR)

In [79]:
print("Total Data in source directory:")
print(len(os.listdir("/content/drive/MyDrive/source/benign")))
print(len(os.listdir("/content/drive/MyDrive/source/malignant")))

Total Data in source directory:
1800
1507


In [85]:
# data = os.listdir("/content/drive/MyDrive/source/benign")
# print(data)

data = os.listdir("/content/drive/MyDrive/source/malignant")
print(data)

['100.jpg', '1000.jpg', '1001.jpg', '1002.jpg', '1004.jpg', '1005.jpg', '1007.jpg', '1008.jpg', '1009.jpg', '101.jpg', '1010.jpg', '1011.jpg', '1012.jpg', '1014.jpg', '1015.jpg', '1016.jpg', '1017.jpg', '102.jpg', '1020.jpg', '1021.jpg', '1022.jpg', '1024.jpg', '1026.jpg', '1027.jpg', '1028.jpg', '103.jpg', '1030.jpg', '1031.jpg', '1032.jpg', '1035.jpg', '1036.jpg', '1037.jpg', '1038.jpg', '1039.jpg', '104.jpg', '1040.jpg', '1041.jpg', '1043.jpg', '1045.jpg', '1046.jpg', '1047.jpg', '1048.jpg', '1049.jpg', '1050.jpg', '1051.jpg', '1052.jpg', '1053.jpg', '1054.jpg', '1056.jpg', '1057.jpg', '1058.jpg', '1059.jpg', '106.jpg', '1060.jpg', '1062.jpg', '1063.jpg', '1064.jpg', '1065.jpg', '1066.jpg', '1067.jpg', '1068.jpg', '1069.jpg', '107.jpg', '1071.jpg', '1072.jpg', '1073.jpg', '1075.jpg', '1076.jpg', '1078.jpg', '1079.jpg', '108.jpg', '1081.jpg', '1082.jpg', '1083.jpg', '1084.jpg', '1085.jpg', '1086.jpg', '1089.jpg', '109.jpg', '1090.jpg', '1091.jpg', '1093.jpg', '1094.jpg', '1095.jpg', 

In [ ]:
import random
import shutil

def split_data(src, train, test, split_size):
  source_data = os.listdir(src)
  source_data = random.sample(source_data, len(source_data))

  for f in enumerate(source_data):
    if f[i]




In [80]:
import random
from shutil import copyfile

def split_data(SOURCE, TRAINING, VALIDATION, SPLIT_SIZE):
  source_data = os.listdir(SOURCE)
  source_data = random.sample(source_data, len(source_data))

  for count, filename in enumerate(source_data):
    if count < SPLIT_SIZE * len(source_data):
      copyfile(SOURCE + filename, TRAINING + filename)
    else:
      copyfile(SOURCE + filename, VALIDATION + filename)

      

BENIGN_SOURCE_DIR = "/content/drive/MyDrive/source/benign"
BENIGN_TRAINING_DIR = "/content/drive/MyDrive/data/training/benign"
BENIGN_VALIDATION_DIR = "/content/drive/MyDrive/data/validation/benign"

MALIGNANT_SOURCE_DIR = "/content/drive/MyDrive/source/malignant"
MALIGNANT_TRAINING_DIR = "/content/drive/MyDrive/data/training/malignant"
MALIGNANT_VALIDATION_DIR = "/content/drive/MyDrive/data/validation/malignant"


split_size = 0.8
split_data(BENIGN_SOURCE_DIR, BENIGN_TRAINING_DIR, BENIGN_VALIDATION_DIR, split_size)
split_data(MALIGNANT_SOURCE_DIR, MALIGNANT_TRAINING_DIR, MALIGNANT_VALIDATION_DIR, split_size)


FileNotFoundError: ignored

In [81]:
print("Total valid Data:")
print(len(os.listdir("/content/drive/MyDrive/data/validation/benign")))
print(len(os.listdir("/content/drive/MyDrive/data/validation/malignant")))

print("Total Training Data")
print(len(os.listdir("/content/drive/MyDrive/data/training/benign")))
print(len(os.listdir("/content/drive/MyDrive/data/training/malignant")))

Total valid Data:
0
0
Total Training Data
0
0


In [ ]:
TRAIN_DIR = "/content/drive/MyDrive/data/train"
TEST_DIR = "/content/drive/MyDrive/data/test"